In [1]:
import csv
import numpy as np
import os

os.getcwd()
dataset1=csv.reader(open('groceries.csv'))

dataset2=[]
#处理dataset 删除dataset中的''元素 并按照字母从a到z排序
for i in dataset1:
    while '' in i:
        i.remove('')
        i.sort()
    dataset2.append(i)
dataset3=dataset2[:]

In [2]:
class treenode:                  #树节点
    def __init__(self,value,number,parent):
        self.value=value         #node name
        self.number=number  #node count
        self.parent=parent     #parent node
        self.children={}          #chlidren node
        self.link=None          #node cross link
    

In [3]:
class confptreenode:                  #树节点
    def __init__(self,value,number,parent):
        self.value=value         #node name
        self.number=number  #node count
        self.parent=parent     #parent node
        self.children={}          #chlidren node    
        
def updateconfptree(row,node,valuenum):                 #row一行数据 node需要更新的树的结点 headtable频繁集 count节点计数
    if row[0] in node.children:                           #如果第一个节点已经是子节点则number加1
        node.children[row[0]].number=node.children[row[0]].number+valuenum
    else:                                                        #如果不存在则建立新的节点 number=1            
        node.children[row[0]]=treenode(row[0],valuenum,node)
    if len(row[1:])>=1:                                         #如果该行数据中还有元素则要递归分配剩下的树
        updateconfptree(row[1:],node.children[row[0]],valuenum)

In [4]:
def subsetsbinary(datasets):    #二进制法求子集   
    N=len(datasets)
    subsets=[]
    for i in range(2**N):        
        subset = []        
        for j in range(N):                   
            if(i>>j )%2==1:                
                subset.append(datasets[j])   
        subsets.append(subset)
    del subsets[0]
    #del subsets[0]
    for i in list(subsets):           #删除所有不包含第一个元素的子集
        if datasets[0] not in i:
            subsets.remove(i)
    return subsets

In [5]:
def updatetree(row,node,headtable):                 #row一行数据 node需要更新的树的结点 headtable频繁集 count节点计数
    if row[0] in node.children:                           #如果第一个节点已经是子节点则number加1
        node.children[row[0]].number=node.children[row[0]].number+1
    else:                                                        #如果不存在则建立新的节点 number=1
        node.children[row[0]]=treenode(row[0],1,node)
        if headtable[row[0]][1]==None:               #headtable节点的link为空,没有连接则建立一个新的link
            headtable[row[0]][1]=node.children[row[0]]
        else:                                                   #存在则在尾部添加
            tmp=headtable[row[0]][1]
            while(tmp.link!=None):
                tmp=tmp.link
            tmp.link=node.children[row[0]]

    #给第一个元素在树中分配好位置后从该行数据集中删除
    #如果该行数据中还有元素则要递归分配剩下的树
    if len(row[1:])>=1:                                         #如果该行数据中还有元素则要递归分配剩下的树
        updatetree(row[1:],node.children[row[0]],headtable)



In [6]:
def createtree(inputdataset,minsupprot):          
    #建立headtable频繁集
    dataset=inputdataset[:]
    headtable = {}
    for row in dataset:                                   #数据集每行
        for item in row:                                   #每行各元素
            if headtable.get(item)==None:           #如果元素在字典中不存在则添加该元素并value为1
                headtable[item]=1
            else:                                              #存在则value加1
                headtable[item]=headtable[item]+1
    if '' in headtable.keys():                            #删除空的数据
        del(headtable[''])
    for item in list(headtable.keys()):               #删除小于minsupport的频繁集
        if headtable[item]<minsupprot: 
            del(headtable[item])
    
    freqitemset=set(headtable.keys())              #频繁项集
    for k in headtable:
        headtable[k]=[headtable[k],None]          #频繁集headtable的尾部加上link
    for trans in dataset:                                #过滤掉support<minsupport的元素
        for item in list(trans):
            if item not in freqitemset:                #删除数据集中不在频繁项集里的元素
                trans.remove(item)
    while [] in dataset:                                  #删除不在频繁集的元素后可能会存在空列表，删除空列表
        dataset.remove([])
    headtable=dict(sorted(headtable.items(), key = lambda x:x[1][0],reverse = True))#对headtable进行从大到小排序

    rootnode=treenode('Null Set',1,None)         #树的根节点
    for row in dataset:
        sortrow=[]
        for headtableitem in headtable:
            if headtableitem in row:
                sortrow.append(headtableitem)
        updatetree(sortrow,rootnode,headtable)
    
    return rootnode,headtable,freqitemset,dataset

In [7]:
def condfptreeonitem(headtable):
    allconfptree={}
    #print(headtable,end='beforesort')
    headtable=dict(sorted(headtable.items(), key = lambda x:x[1][0]))
    #print(headtable,end='aftersort')
    for key,value in headtable.items():    #headtable={value:[num,link]}  #对headtable里的每一个元素进行查找
        allconfptree[key]=[]                          #在condfptree集合里添加该元素的项           
        tmp=value[1]
    #对于链接上的每一个点向上寻找parent直到rootnode
        while(tmp.link!=None):                     #如果不是横向链接的末端向上
            tmp2=tmp
            fptreebaseon={}                          #headtable中链接
            tmpnumber=tmp2.number
            while(tmp2.value!='Null Set'):######################
                fptreebaseon[tmp2.value]=tmpnumber
                tmp2=tmp2.parent
            fptreebaseon[tmp2.value]=1         #Null Set的值设置为1
            #print(tmp2.value,end=' confe\n')
            allconfptree[key].append(fptreebaseon)
            tmp=tmp.link                              #寻找下一个横向链j节点
    #对横向链的末端元素进行parent寻找
        fptreebaseon={}
        tmpnumber=tmp.number
        while(tmp.value!='Null Set'):
            fptreebaseon[tmp.value]=tmpnumber
            tmp=tmp.parent
        fptreebaseon[tmp.value]=1
        allconfptree[key].append(fptreebaseon)    
    return allconfptree

In [8]:
def countfptree(allconfptree,maxleafsize):
    allfrequentdataset={}
    allfrequentdataset1={}
    freqsetonkey=[]
    for key,confptreeonkey in allconfptree.items():      #allconfptree={'key':[value[0],....value[n]]}  value confptree
        tmp={}
        freqset={}
        for tree in confptreeonkey:                 #计算关于key的条件fptree
            for treeitem,num in tree.items():      #计算关于key的confptree中的元素个数
                if treeitem in tmp:
                    tmp[treeitem]=tmp[treeitem]+num
                else:
                    tmp[treeitem]=num
                if 'Null Set' in tmp:
                    del tmp['Null Set']
        for tmpitem,num in tmp.items():
            if num>=maxleafsize:
                freqset[tmpitem]=num
        freqsetonkey.append(freqset)
        #if len(freqset)!=0:#对于长度大于1的项 求出所有的子集，依次计算子集各元素在confptree中同时出现的个数
        subsets=subsetsbinary(list(freqset.keys()))# 子集构成的集合
        for subset in subsets:#对于子集中的每一个集合
            count=0
            lensubset=len(subset)
            for tree in confptreeonkey:
                itemcount=0
                for subsetitem in subset:#如果子集中的项全部出现的频数进行计数
                    if subsetitem in tree:
                        itemcount=itemcount+1
                if itemcount==lensubset:
                    count=count+tree[subsetitem]
            if count>=maxleafsize:
                allfrequentdataset[tuple(subset)]=count
            allfrequentdataset1[tuple(subset)]=count
    return allfrequentdataset,freqsetonkey,allfrequentdataset1

In [9]:
def printconfptree(node):
    if len(node.children)!=0:    #有子集就要输出一个括号
        print('[',end='')
    print('\"',end='')
    print(node.value,end=' ')
    print(node.number,end='')
    if len(node.children)!=0:    #不为0就要输出一个逗号
        print('\", ',end='')
        print('[',end='')
    if len(node.children)==1:
        printconfptree(node.children[list(node.children.keys())[0]])
        print(']',end='')
    if len(node.children)>1:
        i=0
        while(i<len(node.children)-1):
            printconfptree(node.children[list(node.children.keys())[i]])
            i=i+1
            print(',',end=' ')
        printconfptree(node.children[list(node.children.keys())[i]])
        print(']',end='')
    if len(node.children)!=0:    #有子集就要输出一个括号
        print(']',end='')            
        

In [23]:
def constructconfptree(freqsetonkey,allconfptree):
    heightfreqset={}
    for i in freqsetonkey:
        if len(i)>1:
            heightfreqset[list(i.keys())[0]]=list(i.keys())
    contree={}
    for key,values in allconfptree.items():
        if key in list(heightfreqset.keys()):
            contree[key]=confptreenode('Null Set',1,None)
            for row in values:
                newrow=[]
                for rowitem,rowvalue in row.items():
                    if rowitem in heightfreqset[key]:
                        newrow.append(rowitem)
                        count=rowvalue
                newrow.reverse()
                newrow.pop()
                if len(newrow)>0:
                    updateconfptree(newrow,contree[key],count)
    return contree
       
        

In [21]:
def writetocsv(filename,allfrequentdataset):
    with open(filename,"w") as csvFile:
        csvWriter = csv.writer(csvFile)
        for key in allfrequentdataset.keys():
            row='\"{'
            i=0
            while(i<len(key)-1):
                row=row+'\''+key[i]+'\', '
                i+=1
            row=row+'\''+key[i]+'\''        
            row=row+'}\"\n'
            csvFile.write(row)
        csvFile.close

In [11]:
fptree,headtable,freqitemset,aftdataset=createtree(dataset3[:],300)

In [12]:
allconfptree=condfptreeonitem(headtable)

In [13]:
allfrequentdataset,freqsetonkey,allfrequentdataset1=countfptree(allconfptree,300)

In [27]:
writetocsv("submisson.csv",allfrequentdataset)

In [24]:
contree=constructconfptree(freqsetonkey,allconfptree)

In [25]:
for key,value in contree.items(): 
    print("condtional fp-tree on item:",end='')
    print(key)
    printconfptree(value)
    print('')

condtional fp-tree on item:whipped/sour cream
["Null Set 1", ["whole milk 317]]
condtional fp-tree on item:citrus fruit
["Null Set 1", ["whole milk 300]]
condtional fp-tree on item:pastry
["Null Set 1", ["whole milk 327]]
condtional fp-tree on item:sausage
["Null Set 1", ["rolls/buns 301]]
condtional fp-tree on item:tropical fruit
["Null Set 1", ["other vegetables 185, ["whole milk 416", ["other vegetables 168]]]]
condtional fp-tree on item:root vegetables
["Null Set 1", ["other vegetables 238, ["whole milk 481", ["other vegetables 228]]]]
condtional fp-tree on item:bottled water
["Null Set 1", ["whole milk 338]]
condtional fp-tree on item:yogurt
["Null Set 1", [["whole milk 551", [["other vegetables 219", ["rolls/buns 59]], "rolls/buns 94]], "rolls/buns 131, ["other vegetables 208", ["rolls/buns 54]]]]
condtional fp-tree on item:soda
["Null Set 1", ["rolls/buns 236, ["whole milk 394", [["other vegetables 137", ["rolls/buns 43]], "rolls/buns 44]], ["other vegetables 185", ["rolls/buns 

In [26]:
allfrequentdataset


{('onions',): 305,
 ('hygiene articles',): 324,
 ('hamburger meat',): 327,
 ('berries',): 327,
 ('UHT-milk',): 329,
 ('sugar',): 333,
 ('dessert',): 365,
 ('long life bakery product',): 368,
 ('salty snack',): 372,
 ('waffles',): 378,
 ('cream cheese ',): 390,
 ('white bread',): 414,
 ('chicken',): 422,
 ('frozen vegetables',): 473,
 ('chocolate',): 488,
 ('napkins',): 515,
 ('beef',): 516,
 ('curd',): 524,
 ('butter',): 545,
 ('pork',): 567,
 ('coffee',): 571,
 ('margarine',): 576,
 ('frankfurter',): 580,
 ('domestic eggs',): 624,
 ('brown bread',): 638,
 ('whipped/sour cream',): 705,
 ('whipped/sour cream', 'whole milk'): 317,
 ('fruit/vegetable juice',): 711,
 ('pip fruit',): 744,
 ('canned beer',): 764,
 ('newspapers',): 785,
 ('bottled beer',): 792,
 ('citrus fruit',): 814,
 ('citrus fruit', 'whole milk'): 300,
 ('pastry',): 875,
 ('pastry', 'whole milk'): 327,
 ('sausage',): 924,
 ('sausage', 'rolls/buns'): 301,
 ('shopping bags',): 969,
 ('tropical fruit',): 1032,
 ('tropical fr